In [1]:
!pip install -U pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.71.0
    Uninstalling grpcio-1.71.0:
      Successfully uninstalled grpcio-1.71.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires grpcio>=1.71.0, but you have grpcio 1.67.1 which is incompatible.


In [2]:
from pymilvus import MilvusClient, DataType

client = MilvusClient('sample.db')

In [4]:
# from pymilvus import MilvusClient, DataType

# client = MilvusClient('sample.db')

# Define collection schema
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="age", datatype=DataType.INT64, nullable=True) # Nullable field

# Set index params
index_params = client.prepare_index_params()
index_params.add_index(field_name="vector", index_type="IVF_FLAT", metric_type="L2", params={ "nlist": 128 })

# Create collection
client.create_collection(collection_name="user_profiles_null", schema=schema, index_params=index_params)


In [9]:
client.describe_collection

<bound method MilvusClient.describe_collection of <pymilvus.milvus_client.milvus_client.MilvusClient object at 0x786f7af3b110>>

In [10]:
client.describe_index

<bound method MilvusClient.describe_index of <pymilvus.milvus_client.milvus_client.MilvusClient object at 0x786f7af3b110>>

In [11]:
client.describe_database

<bound method MilvusClient.describe_database of <pymilvus.milvus_client.milvus_client.MilvusClient object at 0x786f7af3b110>>

In [13]:
data = [
    {"id": 1, "vector": [0.1, 0.2, 0.3, 0.4, 0.5], "age": 30},
    {"id": 2, "vector": [0.2, 0.3, 0.4, 0.5, 0.6], "age": None},
    {"id": 3, "vector": [0.3, 0.4, 0.5, 0.6, 0.7]}
]

client.insert(collection_name="user_profiles_null", data=data)

{'insert_count': 3, 'ids': [1, 2, 3], 'cost': 0}

In [14]:
res = client.search(
    collection_name="user_profiles_null",
    data=[[0.1, 0.2, 0.4, 0.3, 0.128]],
    limit=2,
    search_params={"params": {"nprobe": 16}},
    output_fields=["id", "age"]
)

print(res)

data: ["[{'id': 1, 'distance': 0.15838398039340973, 'entity': {'age': 30, 'id': 1}}, {'id': 2, 'distance': 0.28278401494026184, 'entity': {'age': 136962888253504, 'id': 2}}]"]


In [15]:
results = client.query(
    collection_name="user_profiles_null",
    filter="age >= 0",
    output_fields=["id", "age"]
)


In Query Null Values are ommited

In [16]:
print(results)

data: ["{'id': 1, 'age': 30}", "{'id': 2, 'age': 136962888253504}", "{'id': 3, 'age': 136962888256576}"]


In [17]:
null_results = client.query(
    collection_name="user_profiles_null",
    filter="", # Query without any filtering condition
    output_fields=["id", "age"],
    limit=10 # `limit` parameter is required when using `query` method without filtering condition
)

Query withhout filter attribute return all data with null values also

In [18]:
print(null_results)

data: ["{'id': 1, 'age': 30}", "{'id': 2, 'age': 136962888253504}", "{'id': 3, 'age': 136962888256576}"]


default_value parameter

In [3]:
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="age", datatype=DataType.INT64, default_value=18)
schema.add_field(field_name="status", datatype=DataType.VARCHAR, default_value="active", max_length=10)

index_params = client.prepare_index_params()
index_params.add_index(field_name="vector", index_type="IVF_FLAT", metric_type="L2", params={ "nlist": 128 })

client.create_collection(collection_name="user_profiles_default", schema=schema, index_params=index_params)


In [ ]:
data = [
    {"id": 1, "vector": [0.1, 0.2,0.0,0.0,0.128], "age": 30, "status": "premium"},
    {"id": 2, "vector": [0.2, 0.3,0.0,0.0,0.129]},
    {"id": 3, "vector": [0.3, 0.4,0.0,0.0,0.130], "age": 25, "status": None},
    {"id": 4, "vector": [0.4, 0.5,0.0,0.0,0.131], "age": None, "status": "inactive"}
]

client.insert(collection_name="user_profiles_default", data=data)